In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
import urllib.request
import re
%pprint
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 attempt_to_pickle	 example_iterrows	 get_column_descriptions	 
load_csv	 load_dataframes	 load_object	 math	 os	 pd	 pickle	 re	 save_dataframes	 
store_objects	 sys	 urllib	 



# Scraping the Building Stats

In [2]:

from bs4 import BeautifulSoup

wiki_url = 'https://ageofempires.fandom.com'
def get_page_soup(page_url):
    with urllib.request.urlopen(page_url) as response:
        page_html = response.read()
    page_soup = BeautifulSoup(page_html, 'html.parser')
    
    return page_soup

In [3]:

def get_content_text_div(page_url):
    page_soup = get_page_soup(page_url)
    content_text_selector = '#mw-content-text'
    content_text_div = page_soup.select(content_text_selector)[0]
    
    return content_text_div

In [4]:

main_page_url = '{}/wiki/Buildings_(Age_of_Empires_II)'.format(wiki_url)
main_content_text_div = get_content_text_div(main_page_url)

In [5]:

import numpy as np

# Get the DOM
buildings_table = main_content_text_div.find_all('table')[1]
columns_tr = buildings_table.find_all('tr')[0]
columns_abbr_list = columns_tr.find_all('abbr')
tr_list = buildings_table.find_all('tr')

In [9]:

columns_dict = {columns_abbr.text: columns_abbr['title'] for columns_abbr in columns_abbr_list}
store_objects(building_columns_dict=columns_dict)

Pickling to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\pickle\building_columns_dict.pickle


In [10]:

# Build the df
columns_list = [abbr['title'] for abbr in columns_abbr_list]
buildings_df = pd.read_html(str(buildings_table), header=0)[0]
buildings_df = buildings_df.set_index('Building')
buildings_df.index.name = 'Building'
buildings_df.columns = columns_list
buildings_df = buildings_df.iloc[:-1]

# Fix the age column
age_list = []
link_dict = {}
for tr in tr_list[1:-1]:
    td_list = tr.find_all('td')
    building_td = td_list[0]
    age_td = td_list[1]
    building_link = building_td.find_all('a')[1]
    age_link = age_td.find_all('a')[0]
    age_list.append(age_link['title'])
    building_name = building_link['title'].split('(')[0].strip()
    link_dict[building_name] = building_link['href']
buildings_df['Age'] = pd.Series(age_list, index=buildings_df.index)
buildings_df

,Age,Hit points,Garrison capacity,Armor,Pierce armor,Line of Sight,Range,Attack,Wood cost,Stone cost,Gold cost,Construction time in seconds,Size in tiles
Building,,,,,,,,,,,,,
Town Center,Castle Age,2400,15,6.0,8.0,8.0,6.0,5.0,275.0,100,0.0,150,4x4
House,Dark Age,900,0,3.0,10.0,2.0,0.0,0.0,25.0,0,0.0,25,2x2
Dock,Dark Age,1800,10,3.0,10.0,6.0,0.0,0.0,150.0,0,0.0,35,3x3
Fish Trap,Dark Age,50,0,0.0,0.0,1.0,0.0,0.0,100.0,0,0.0,40,1x1
Mill,Dark Age,1000,0,3.0,10.0,6.0,0.0,0.0,100.0,0,0.0,35,2x2
Farm,Dark Age,480,0,0.0,0.0,1.0,0.0,0.0,60.0,0,0.0,15,3x3
Market,Feudal Age,2100,0,3.0,10.0,6.0,0.0,0.0,175.0,0,0.0,60,4x4
Lumber Camp,Dark Age,1000,0,3.0,10.0,6.0,0.0,0.0,100.0,0,0.0,35,2x2
Mining Camp,Dark Age,1000,0,3.0,10.0,6.0,0.0,0.0,100.0,0,0.0,35,2x2


In [11]:

link_dict

{'Town Center': '/wiki/Town_Center_(Age_of_Empires_II)', 'House': '/wiki/House_(Age_of_Empires_II)', 'Dock': '/wiki/Dock_(Age_of_Empires_II)', 'Fish Trap': '/wiki/Fish_Trap', 'Mill': '/wiki/Mill_(Age_of_Empires_II)', 'Farm': '/wiki/Farm_(Age_of_Empires_II)', 'Market': '/wiki/Market_(Age_of_Empires_II)', 'Lumber Camp': '/wiki/Lumber_Camp_(Age_of_Empires_II)', 'Mining Camp': '/wiki/Mining_Camp_(Age_of_Empires_II)', 'Barracks': '/wiki/Barracks_(Age_of_Empires_II)', 'Archery Range': '/wiki/Archery_Range_(Age_of_Empires_II)', 'Stable': '/wiki/Stable_(Age_of_Empires_II)', 'Blacksmith': '/wiki/Blacksmith', 'Siege Workshop': '/wiki/Siege_Workshop_(Age_of_Empires_II)', 'Monastery': '/wiki/Monastery_(Age_of_Empires_II)', 'University': '/wiki/University', 'Castle': '/wiki/Castle_(Age_of_Empires_II)', 'Palisade Wall': '/wiki/Palisade_Wall', 'Palisade Gate': '/wiki/Palisade_Gate', 'Stone Wall': '/wiki/Stone_Wall_(Age_of_Empires_II)', 'Gate': '/wiki/Gate_(Age_of_Empires_II)', 'Fortified Wall': '/wik

In [12]:

space_regex = re.compile(r'\s+')
def process_small_list_of_units(appl_civ_text, row_dict):
    appl_civ_list = space_regex.split(appl_civ_text)
    for civ_name in civ_name_list:
        row_dict[civ_name] = 0
    for civ_name in appl_civ_list:
        if civ_name in civ_name_list:
            row_dict[civ_name] = 1
    
    return row_dict

In [13]:

toc_selector = '#toc'
def process_exceptions(link_soup, row_dict):
    desc_p_list = link_soup.select(toc_selector)
    if len(desc_p_list):
        for sib in desc_p_list[0].previous_siblings:
            if 'available to all civilizations except' in str(sib):
                break
        process_all_except(sib.text.strip(), row_dict)
    
    return row_dict

In [14]:

def process_article(art_ul_list, row_dict):

    # Get available civs
    available_ul = art_ul_list[0]
    for link in available_ul.find_all('a'):
        civ_name = link['title'].split('(')[0].strip()
        row_dict[civ_name] = 1

    # Get unavailable civs
    unavailable_ul = art_ul_list[1]
    for link in unavailable_ul.find_all('a'):
        civ_name = link['title'].split('(')[0].strip()
        row_dict[civ_name] = 0

    return row_dict

In [15]:

def process_all_except(appl_civ_text, row_dict):
    for civ_name in civ_name_list:
        if civ_name in appl_civ_text:
            row_dict[civ_name] = 0
        else:
            row_dict[civ_name] = 1
    
    return row_dict

In [16]:

article_selector = '#mw-content-text > table.article-table'
civ_name_list = load_object('civ_name_list')
def process_unit(link_soup, appl_civ_text, row_dict):
    for column_name in civ_name_list:
        columns_set.add(column_name)
    if 'All civilizations' in appl_civ_text:
        for civ_name in civ_name_list:
            row_dict[civ_name] = 1
    elif 'All except ' in appl_civ_text:
        row_dict = process_all_except(appl_civ_text, row_dict)
    elif 'See ' in appl_civ_text:
        art_table_list = link_soup.select(article_selector)
        if len(art_table_list):
            art_table = art_table_list[0]
            art_ul_list = art_table.find_all('ul')
            if len(art_ul_list) == 2:
                row_dict = process_article(art_ul_list, row_dict)
            else:
                row_dict = process_exceptions(link_soup, row_dict)
        else:
            row_dict = process_exceptions(link_soup, row_dict)
    else:
        row_dict = process_small_list_of_units(appl_civ_text, row_dict)
    
    return row_dict

In [23]:

def consume_aside(unit_soup, row_dict):
    aside_list = unit_soup.select('aside')
    for aside_soup in aside_list:
        for data_source_div in aside_soup.find_all('div', {'data-source': True}):
            data_source_h3_list = data_source_div.find_all('h3')
            if len(data_source_h3_list):
                h3_siblings_list = list(data_source_h3_list[0].next_siblings)
                if len(h3_siblings_list):
                    column_name = data_source_div['data-source']
                    full_name = data_source_div.find_all('h3')[0]
                    link_list = full_name.find_all('a')
                    if len(link_list):
                        full_name = link_list[0].text
                    else:
                        full_name = full_name.text
                    columns_dict[column_name] = full_name
                    value_list = list(h3_siblings_list[1].strings)
                    column_value = ' '.join(x.strip() for x in value_list).strip()
                    if column_name == 'Available':
                        row_dict = process_unit(unit_soup, column_value, row_dict)
                    else:
                        columns_set.add(column_name)
                        row_dict[column_name] = column_value
    
    return row_dict

In [24]:

rows_list = []
index_list = []
columns_set = set(buildings_df.columns)
for building_name, building_url in link_dict.items():
    page_url = '{}{}'.format(wiki_url, building_url)
    content_text_div = get_content_text_div(page_url)
    building_name = building_name.split('(')[0].strip()
    if building_name not in index_list:
        index_list.append(building_name)
        rows_list.append(consume_aside(content_text_div, {'building_url': page_url}))

In [25]:

store_objects(building_columns_dict=columns_dict)
columns_dict

Pickling to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\pickle\building_columns_dict.pickle


{'A': 'Age', 'HP': 'Hit points', 'Ga': 'Garrison capacity', 'Ar': 'Armor', 'PA': 'Pierce armor', 'LOS': 'Line of Sight', 'Ra': 'Range', 'At': 'Attack', 'W': 'Wood cost', 'S': 'Stone cost', 'G': 'Gold cost', 'CT': 'Construction time in seconds', 'Sz': 'Size in tiles', 'Type': 'Type', 'Intro': 'Introduced in', 'Available': 'Civilization', 'Age': 'Age', 'Time': 'Construction time', 'Use': 'Use', 'Wood': 'Wood', 'Stone': 'Stone', 'Size': 'Size', 'Garrison': 'Garrison', 'Attack': 'Attack', 'BAttack': 'Attack bonuses', 'ROF': 'Rate of Fire', 'Range': 'Range', 'Accuracy': 'Accuracy', 'PSpeed': 'Projectile speed', 'Armor': 'Melee armor', 'PArmor': 'Pierce armor', 'Class': 'Armor classes', 'Required': 'Required building', 'Next': 'Required for', 'MinRange': 'Minimum range', 'UpTo': 'Upgraded to', 'UpCost': 'Upgrade cost', 'UpTime': 'Upgrade time', 'UpFrom': 'Upgraded from', 'Gold': 'Gold', 'Research': 'Required research'}

In [80]:

other_columns_set = columns_set - set(civ_name_list)
other_columns_set = other_columns_set - set(['Heading'])
columns_list = ['Heading'] + list(other_columns_set) + civ_name_list
building_df = pd.DataFrame(data=rows_list, index=index_list,
                             columns=columns_list).dropna(axis='columns', how='all')

In [81]:

building_df.sample(5).T.sample(5)

,Town Center,Watch Tower,Stone Wall,House,Farm
Range,6,8,NaN,NaN,NaN
Use,Create and improve Villagers Advance Ages Drop...,Attacks enemies at sight,Delay enemy units,Provides population,Provides food
Vikings,1,1,1,1,1
Celts,1,1,1,1,1
Franks,1,1,1,1,1


In [159]:

dataframes_dict = load_dataframes(building_df='building_df')
index_name = 'Building'
building_df = dataframes_dict['building_df']
if 'Unnamed: 0' in building_df.columns:
    building_df = building_df.set_index('Unnamed: 0')
elif index_name in building_df.columns:
    building_df = building_df.set_index(index_name)
building_df.index.name = index_name

Attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\pickle\building_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\csv\building_df.csv.


In [130]:

columns_list = ['Time', 'Wood', 'Stone', 'Gold']
for cn in columns_list:
    building_df[cn] = building_df[cn].fillna(0)
    building_df[cn] = building_df[cn].map(lambda x: int(re.sub(r'\D+', '', str(x))))

Attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\pickle\building_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\csv\building_df.csv.


In [160]:

building_df.index.tolist()

['Town Center', 'Siege Workshop', 'Monastery', 'University', 'Castle', 'Fortified Wall', 'Guard Tower', 'Harbor', 'House', 'Dock', 'Mill', 'Farm', 'Lumber Camp', 'Mining Camp', 'Barracks', 'Palisade Wall', 'Palisade Gate', 'Outpost', 'Fish Trap', 'Market', 'Archery Range', 'Stable', 'Blacksmith', 'Stone Wall', 'Gate', 'Watch Tower', 'Gate (Fortified)', 'Keep', 'Bombard Tower', 'Wonder', 'Feitoria']

In [161]:

building_df.columns.tolist()

['Age', 'HP', 'HP_upon_reaching_Dark_Age', 'HP_upon_reaching_Feudal_Age', 'HP_upon_reaching_Castle_Age', 'HP_upon_reaching_Imperial_Age', 'Range', 'Required', 'Stone', 'Wood', 'Class', 'LOS', 'ROF', 'Time', 'PSpeed', 'Attack', 'Size', 'Gold', 'UpTime', 'BAttack', 'UpFrom', 'Research', 'Intro', 'Type', 'MinRange', 'Use', 'Armor', 'Next', 'Accuracy', 'Garrison', 'PArmor', 'UpCost', 'UpTo', 'Aztecs', 'Berbers', 'Britons', 'Burmese', 'Byzantines', 'Celts', 'Chinese', 'Ethiopians', 'Franks', 'Goths', 'Huns', 'Incas', 'Indians', 'Italians', 'Japanese', 'Khmer', 'Koreans', 'Magyars', 'Malay', 'Malians', 'Mayans', 'Mongols', 'Persians', 'Portuguese', 'Saracens', 'Slavs', 'Spanish', 'Teutons', 'Turks', 'Vietnamese', 'Vikings']

In [162]:

# Buildings have +10%/+20%/+30%/+40% HP in the Dark/Feudal/Castle/Imperial Age.
building_df['HP'] = building_df['HP'].map(lambda x: re.sub(r',', '', str(x)))
building_df['HP'] = building_df['HP'].map(lambda x: re.sub(r' +', ' ', str(x)))
building_df['HP'].unique()

array(['2400', '1800 2100', '2100', '4800', '3000', '1500', '2000',
       '550 750 900 900', '1800', '600 800 1000 1000', '480',
       '1200 1500 1800 2100', '250', '400', '500', '50', '1800 2100 2100',
       '1500 1800 2100', '900 1800 1800', '1375 2750 2750', '1020',
       '1375 4000 4000', '2250', '2220', '5200'], dtype=object)

In [163]:

columns_list = ['Age', 'HP']
building_df[columns_list].sort_values('Age')

,Age,HP
Building,,
Town Center,Castle Age,2400
Siege Workshop,Castle Age,1800 2100
Monastery,Castle Age,2100
University,Castle Age,2100
Castle,Castle Age,4800
Fortified Wall,Castle Age,3000
Guard Tower,Castle Age,1500
Harbor,Castle Age,2000
Outpost,Dark Age,500


In [164]:

age_list = ['Dark', 'Feudal', 'Castle', 'Imperial']
for i, age in enumerate(age_list):
    mask_series = building_df['Age'] == '{} Age'.format(age)
    for j in range(i, len(age_list)):
        cn = 'HP_upon_reaching_{}_Age'.format(age_list[j])
        
        def f(x):
            hp_list = str(x).split(' ')

            return int(hp_list[min(len(hp_list)-1, j-i)])
    
        building_df.loc[mask_series, cn] = building_df.loc[mask_series, 'HP'].map(f)

In [165]:

xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
file_path = os.path.join(xlsx_folder, 'building_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
building_df.to_excel(file_path)

Saving to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\data\xlsx\building_df.xlsx


In [156]:

file_path = os.path.join(DATA_CSV_FOLDER, 'building_df.csv')
print('Saving to {}'.format(os.path.abspath(file_path)))
building_df.to_csv(file_path)

Saving to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\data\csv\building_df.csv


In [157]:

save_dataframes(include_index=True, building_df=building_df)

Saving to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\csv\building_df.csv
